# From-scratch Implementation

## Import Libraries
Load các thư viện cần thiết

In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## Import data
Load data vào để xử lý

In [39]:
data = pd.read_csv('data.txt',delim_whitespace=True,header=None)

In [40]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [41]:
X_train = np.c_[data.iloc[:400,:-1].values,np.ones((400,1))] # Append '1' column for computing with bias
Y_train = data.iloc[:400,-1].values.reshape(400,1)
X_test =  np.c_[data.iloc[400:,:-1].values,np.ones((106,1))]
Y_test = data.iloc[400:,-1].values.reshape(106,1)

## Shapes
Thống nhất shape của các ma trận
<br> Shape of W = `(n,1)` # n đã tính cột 1 được thêm vào
<br>Shape of X = `(m,n)` 
<br>Shape of Y = `(m,1)`

## Randomly Initialize Weights
Khởi tạo các trọng số bằng ngẫu nhiên theo phân phối chuẩn

In [42]:
def initWeightsByRandom(X):
    n = X.shape[1]
    W = np.random.normal(size=(n,1))
    return W

## Mini Batches Generator
Hàm tạo thành các minibatches từ tập X,Y

In [43]:
def createMiniBatches(X,Y,batchSize=32):
    random_state = np.random.permutation(X.shape[0])
    X = X[random_state] #Shuffle X
    Y = Y[random_state] #Shuffle Y
    X_batches = []
    Y_batches = []
    for i in range(0,X.shape[0],batchSize):
        if i+batchSize > X.shape[0]: # Nếu tới minibatch cuối cùng
            X_batches.append(X[i:]) # Append toàn bộ phần còn lại vào mini cuối và break
            Y_batches.append(Y[i:])
            break
        X_batches.append(X[i:i+batchSize])
        Y_batches.append(Y[i:i+batchSize])
    return X_batches,Y_batches

## Prediction
Tính đầu ra dựa trên X và W (hiện tại) 
$$H(x) = X * W^T$$

In [44]:
def Predict(X,W):
    return X.dot(W)

## Compute Cost
Tính chi phí/ độ lỗi của mô hình thông qua X,Y và W 

In [45]:
def Cost(X,Y,W):
    m = X.shape[0]
    return 1/(2*m)*np.sum((Predict(X,W) - Y)**2)
    

## Compute Gradient
Tìm gradient ứng với mỗi theta (vectorization)
$$\frac{1}{m}X^T(\hat{Y}-Y)$$

In [46]:
def Compute_Grad(X,Y,W):
    m = X.shape[0]
    Grad = 1/m*(X.T).dot(Predict(X,W)-Y)
    return Grad

## Optimizer
Tối ưu W bằng cách cập nhật lại với gradient tìm được
$$W = W - \alpha*\frac{\partial{L}}{\partial{W}}$$

In [47]:
def GradientDescentOptimizer(X,Y,W,learning_rate,decay_rate):
    Grad = Compute_Grad(X,Y,W)
    W = W - decay_rate*learning_rate*Grad
    return W

## Combine functions to make full model
Phối hợp các hàm để tạo thành 1 mô hình hoàn chỉnh

In [54]:
def Model(X,Y,X_test,Y_test,learning_rate,batchSize=32,epochs=10000, printCostTrain = False, printCostDev = False):
    W = initWeightsByRandom(X)
    Scaler = MinMaxScaler().fit(X)
    X = Scaler.transform(X)
    X_test = Scaler.transform(X_test)
    decay_rate = 1
    for i in range(epochs):
        X_batches, Y_batches = createMiniBatches(X,Y,batchSize=batchSize)
        if (i%750==0):
            decay_rate/=2
        for X_batch,Y_batch in zip(X_batches, Y_batches):
            W = GradientDescentOptimizer(X_batch,Y_batch,W,learning_rate,decay_rate = decay_rate)
        if printCostTrain:
            print('Train Cost:',end='')
            print(Cost(X,Y,W))
        if printCostDev:
            print('Dev Cost:', end='')
            print(Cost(X_test,Y_test,W))
    

## Train your model
Đưa data vào và huấn luyện mô hình

In [55]:
Model(X_train,Y_train,X_test,Y_test, printCostTrain=True, batchSize = 32,epochs = 10000, learning_rate=0.2)

Train Cost:50.22001753696097
Train Cost:41.68650181112802
Train Cost:36.474002158682445
Train Cost:32.78218603740729
Train Cost:30.33071672784538
Train Cost:28.969444561662048
Train Cost:26.682705103289038
Train Cost:25.48593086091487
Train Cost:24.22356923738076
Train Cost:23.22023839341383
Train Cost:22.392946585486406
Train Cost:21.59400859869109
Train Cost:20.950200941709408
Train Cost:20.351934258321823
Train Cost:19.89918623568713
Train Cost:19.702692090860918
Train Cost:19.246616704439784
Train Cost:18.506048541359633
Train Cost:18.17191522129777
Train Cost:17.837761062278254
Train Cost:17.631715844546836
Train Cost:17.257667559012162
Train Cost:16.998219073916694
Train Cost:16.894921973861855
Train Cost:16.569478407613534
Train Cost:16.41088921029852
Train Cost:16.18243329168742
Train Cost:16.012408266908366
Train Cost:15.924297202670619
Train Cost:15.834136065185437
Train Cost:15.576659819886654
Train Cost:15.714667180011865
Train Cost:15.389310324631545
Train Cost:15.48650525

Train Cost:13.27813217245996
Train Cost:13.222099849774777
Train Cost:13.464976326383969
Train Cost:13.367574996481885
Train Cost:13.22370026445634
Train Cost:13.38614315658621
Train Cost:13.211792310550495
Train Cost:13.220725534693582
Train Cost:13.210094872413906
Train Cost:13.260172334968384
Train Cost:13.538557146386362
Train Cost:13.205642956558782
Train Cost:13.262661421033558
Train Cost:13.260879599276798
Train Cost:13.349242725195069
Train Cost:13.205648909880356
Train Cost:13.21601822831275
Train Cost:13.224830890434278
Train Cost:13.215131265781338
Train Cost:13.229710623746278
Train Cost:13.200911906084173
Train Cost:13.201114878849626
Train Cost:13.330596118190854
Train Cost:13.324942482376446
Train Cost:13.744916150832564
Train Cost:13.224076829057015
Train Cost:13.208363291849048
Train Cost:13.21827390723116
Train Cost:13.255164699413601
Train Cost:13.229858444760817
Train Cost:13.200566220886431
Train Cost:13.201659464682654
Train Cost:13.205111880270762
Train Cost:13.2

Train Cost:13.18139010454578
Train Cost:13.222863857626026
Train Cost:13.19841210718123
Train Cost:13.184974481744444
Train Cost:13.18056881476024
Train Cost:13.17880535205024
Train Cost:13.196866364474046
Train Cost:13.18363334453204
Train Cost:13.1772198975837
Train Cost:13.201394528719698
Train Cost:13.179164912198223
Train Cost:13.183337511695854
Train Cost:13.176369513394944
Train Cost:13.176287824132245
Train Cost:13.177097852852812
Train Cost:13.189339904041272
Train Cost:13.204852871399817
Train Cost:13.202288749074668
Train Cost:13.177763854890584
Train Cost:13.181757112901835
Train Cost:13.177669437959151
Train Cost:13.197630234807125
Train Cost:13.17782745413944
Train Cost:13.179918032703702
Train Cost:13.176963214906309
Train Cost:13.184022642173204
Train Cost:13.177076299344822
Train Cost:13.198023623313674
Train Cost:13.183061187677609
Train Cost:13.20849737893774
Train Cost:13.185688554347823
Train Cost:13.185198619499065
Train Cost:13.187867852780023
Train Cost:13.18072

Train Cost:13.192463589098328
Train Cost:13.287492052011906
Train Cost:13.175948735256698
Train Cost:13.175203203257619
Train Cost:13.176624414520589
Train Cost:13.174864526584242
Train Cost:13.17596592188972
Train Cost:13.176175519456415
Train Cost:13.175662588122405
Train Cost:13.19892126752201
Train Cost:13.181427653091504
Train Cost:13.178968326634305
Train Cost:13.207770589563436
Train Cost:13.173644701553554
Train Cost:13.176425115892403
Train Cost:13.175107291257309
Train Cost:13.204086965153977
Train Cost:13.174189651653805
Train Cost:13.177979818463376
Train Cost:13.20580777654256
Train Cost:13.205480907034962
Train Cost:13.174333843017248
Train Cost:13.182199117259902
Train Cost:13.174857183344907
Train Cost:13.199084428337665
Train Cost:13.321637378844443
Train Cost:13.189186308039503
Train Cost:13.182426989631614
Train Cost:13.181702190127849
Train Cost:13.1787955124434
Train Cost:13.208104610244357
Train Cost:13.182723390430608
Train Cost:13.196140843967296
Train Cost:13.1

Train Cost:13.173707933299166
Train Cost:13.173975514451282
Train Cost:13.173105992147539
Train Cost:13.177363170671283
Train Cost:13.174541227116507
Train Cost:13.17363542134897
Train Cost:13.173087056737955
Train Cost:13.173537429444128
Train Cost:13.173325134767115
Train Cost:13.17380621183393
Train Cost:13.176254911788352
Train Cost:13.177264188438757
Train Cost:13.175098382482693
Train Cost:13.179120079373313
Train Cost:13.181894918653507
Train Cost:13.173341282561424
Train Cost:13.173348584413562
Train Cost:13.173706231367483
Train Cost:13.172952281091618
Train Cost:13.172889770851759
Train Cost:13.181142441499842
Train Cost:13.173595580686374
Train Cost:13.17363685210002
Train Cost:13.172992591024077
Train Cost:13.177218155711964
Train Cost:13.17423976771755
Train Cost:13.184133398215463
Train Cost:13.173448617111694
Train Cost:13.173032468820436
Train Cost:13.17470333655598
Train Cost:13.17305208560143
Train Cost:13.172802760128151
Train Cost:13.175856733913974
Train Cost:13.17

...

Train Cost:13.172218839977868
Train Cost:13.172218890139124
Train Cost:13.172218778338063
Train Cost:13.172219029829211
Train Cost:13.1722191417556
Train Cost:13.172219493278803
Train Cost:13.172219473043722
Train Cost:13.172219611864085
Train Cost:13.172219711389193
Train Cost:13.172219270220262
Train Cost:13.172219384871699
Train Cost:13.172219839809593
Train Cost:13.172220235567421
Train Cost:13.172220167984966
Train Cost:13.172220386089675
Train Cost:13.172219933667563
Train Cost:13.17222023699983
Train Cost:13.172220050866834
Train Cost:13.172220011631913
Train Cost:13.172220073662702
Train Cost:13.172219905547923
Train Cost:13.17222011131126
Train Cost:13.172220259785995
Train Cost:13.172220419060393
Train Cost:13.172219412159277
Train Cost:13.172218881474496
Train Cost:13.172218931933884
Train Cost:13.172218927074882
Train Cost:13.172218562165739
Train Cost:13.172218678854644
Train Cost:13.17221873485296
Train Cost:13.172218919800011
Train Cost:13.172218867532127
Train Cost:13.1

Train Cost:13.172218734342446
Train Cost:13.172218748234723
Train Cost:13.17221878982805
Train Cost:13.172218649039147
Train Cost:13.172218617975846
Train Cost:13.172218643954565
Train Cost:13.172218663073044
Train Cost:13.172218685043728
Train Cost:13.172218666260942
Train Cost:13.172218682283999
Train Cost:13.172218675365066
Train Cost:13.172218520112265
Train Cost:13.172218345495878
Train Cost:13.172218286101563
Train Cost:13.172218252516869
Train Cost:13.172218322158695
Train Cost:13.172218208120073
Train Cost:13.172218110720266
Train Cost:13.172218161774145
Train Cost:13.17221815426215
Train Cost:13.172218161108233
Train Cost:13.172218209095968
Train Cost:13.172218144803379
Train Cost:13.172218150889726
Train Cost:13.172218077982448
Train Cost:13.17221802663508
Train Cost:13.172217991377302
Train Cost:13.172217982738095
Train Cost:13.17221794756238
Train Cost:13.17221793312027
Train Cost:13.172217918398047
Train Cost:13.172217864984455
Train Cost:13.172217857869876
Train Cost:13.1

Train Cost:13.17221800298337
Train Cost:13.172218020160104
Train Cost:13.17221809618648
Train Cost:13.172218044774759
Train Cost:13.172218067727188
Train Cost:13.172218082482983
Train Cost:13.172218108899315
Train Cost:13.172218159350969
Train Cost:13.17221812264748
Train Cost:13.17221806830697
Train Cost:13.172218019727147
Train Cost:13.172218016667399
Train Cost:13.172218005553063
Train Cost:13.172218109237289
Train Cost:13.172218103501628
Train Cost:13.172218086352894
Train Cost:13.17221810901636
Train Cost:13.172218120940126
Train Cost:13.17221809637719
Train Cost:13.172218102842653
Train Cost:13.172218076681297
Train Cost:13.172218053126375
Train Cost:13.17221809102514
Train Cost:13.172218042848439
Train Cost:13.172218092872459
Train Cost:13.172218110291176
Train Cost:13.172218107042022
Train Cost:13.17221807093354
Train Cost:13.172218049425746
Train Cost:13.172218116553841
Train Cost:13.172218152500053
Train Cost:13.172218187625035
Train Cost:13.172218148047614
Train Cost:13.1722

Train Cost:13.172217728358167
Train Cost:13.17221769030441
Train Cost:13.172217668932513
Train Cost:13.172217683325277
Train Cost:13.172217667586832
Train Cost:13.172217670026455
Train Cost:13.172217706075884
Train Cost:13.172217708544903
Train Cost:13.172217752561053
Train Cost:13.172217757228227
Train Cost:13.172217764824056
Train Cost:13.172217753364615
Train Cost:13.172217741345971
Train Cost:13.172217759856817
Train Cost:13.17221777971717
Train Cost:13.172217782063441
Train Cost:13.172217753851625
Train Cost:13.172217727943947
Train Cost:13.172217730074507
Train Cost:13.172217753065423
Train Cost:13.17221778134575
Train Cost:13.1722176999762
Train Cost:13.172217697469634
Train Cost:13.172217682111132
Train Cost:13.172217706548254
Train Cost:13.172217666422794
Train Cost:13.172217666073673
Train Cost:13.172217680802595
Train Cost:13.172217693039858
Train Cost:13.172217656112462
Train Cost:13.172217725762357
Train Cost:13.172217716658599
Train Cost:13.172217694194973
Train Cost:13.1

Train Cost:13.1722176140751
Train Cost:13.172217627545987
Train Cost:13.172217621475939
Train Cost:13.172217634710776
Train Cost:13.17221764422722
Train Cost:13.172217644349068
Train Cost:13.172217653708362
Train Cost:13.172217654096956
Train Cost:13.172217649480315
Train Cost:13.172217649968914
Train Cost:13.172217646083272
Train Cost:13.172217654938965
Train Cost:13.17221765812862
Train Cost:13.172217648194271
Train Cost:13.172217652619922
Train Cost:13.17221765821055
Train Cost:13.172217651979622
Train Cost:13.172217640991965
Train Cost:13.172217635427733
Train Cost:13.172217631630097
Train Cost:13.172217640932267
Train Cost:13.172217669244647
Train Cost:13.172217662549677
Train Cost:13.172217653013641
Train Cost:13.17221763902029
Train Cost:13.172217629431776
Train Cost:13.172217624635255
Train Cost:13.172217611098256
Train Cost:13.172217603649228
Train Cost:13.172217602531685
Train Cost:13.172217592746193
Train Cost:13.172217592094835
Train Cost:13.172217616448796
Train Cost:13.17

# Machine Learning with scikitlearn

## Import Libraries
Import các thư viện cần thiết 

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Split data to training set & test set
Chia data thành training/test set để đánh giá sau đó

In [91]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=2)

## Train/Fit your model
Huấn luyện mô hình dựa trên tập train

In [96]:
model = LinearRegression().fit(X_train,Y_train)

## Evaluation
Đánh giá mô hình dựa trên mean squared error dựa trên training set và test set

In [93]:
Y_pred_train = model.predict(X_train)
Y_pred_test = model.predict(X_test)
print('Mean squared error on training set: ',mean_squared_error(Y_train,Y_pred_train))
print('Mean squared error on test set: ',mean_squared_error(Y_test,Y_pred_test))

Mean squared error on training set:  22.95369357611288
Mean squared error on test set:  18.495420122448575


Vậy là chúng ta đã có mô hình khá ốn rồi đấy! error chấp nhận được trên cả tập training và test set